In [7]:
import numpy as np
import matplotlib.pyplot as plt
import sympy as sm
from scipy.constants import m_e,m_p,m_n

from utils import timeit
from forcefield import LennardJonesPotential, construct_param_matrix
from simpleMD import compute_Hem
import constants

The Jacobi of the dipole function has the form:
\begin{align*}
    \begin{bmatrix}
    \partial \mu_x / \partial r_x & \partial \mu_y / \partial r_x & \partial \mu_z / \partial r_x 
    \\
    \partial \mu_x / \partial r_y & \partial \mu_y / \partial r_y & \partial \mu_z / \partial r_y 
    \\
    \partial \mu_x / \partial r_z & \partial \mu_y / \partial r_z & \partial \mu_z / \partial r_z
    \end{bmatrix}
\end{align*}

In [2]:
sm_Rax = sm.symbols("Rax")
sm_Ray = sm.symbols("Ray")
sm_Raz = sm.symbols("Raz")
sm_Ra = [sm_Rax, sm_Ray, sm_Raz]

sm_Rbx = sm.symbols("Rbx")
sm_Rby = sm.symbols("Rby")
sm_Rbz = sm.symbols("Rbz")
sm_Rb = [sm_Rbx, sm_Rby, sm_Rbz]

sm_mu0 = sm.symbols("mu0")
sm_a = sm.symbols("a")
sm_d0 = sm.symbols("d0")
sm_d1 = sm.symbols("d1")
sm_d2 = sm.symbols("d2")
sm_d7 = sm.symbols("d7")

class BaseDipoleFunction:
    def __init__(self):
        
        self.dipole_function = [
            sm.lambdify(
            [sm_Rax, sm_Ray, sm_Raz, sm_Rbx, sm_Rby, sm_Rbz, 
             ] + self.parameters_sm, 
                dipole_exp
        ) for dipole_exp in self.dipole_exp]
        
        self.generate_Jacobi()
        #self.generate_Hessian()

    def generate_Jacobi(self):
        self.Ja_exp = [[],[],[]]
        self.Ja_func = [[],[],[]]

        self.Jb_exp = [[],[],[]]
        self.Jb_func = [[],[],[]]

        for i,Ra in enumerate(sm_Ra):
            for j,dipole_f in enumerate(self.dipole_exp):
                d_mu = sm.diff(dipole_f, Ra)
                self.Ja_exp[i].append(d_mu)
                self.Ja_func[i].append(sm.lambdify(
                    [sm_Rax, sm_Ray, sm_Raz, sm_Rbx, sm_Rby, sm_Rbz, 
                    ]+ self.parameters_sm, d_mu
                ))

                d_mu = sm.diff(dipole_f, sm_Rb[i])
                self.Jb_exp[i].append(d_mu)
                self.Jb_func[i].append(sm.lambdify(
                    [sm_Rax, sm_Ray, sm_Raz, sm_Rbx, sm_Rby, sm_Rbz, 
                    ]+ self.parameters_sm, d_mu
                ))

    def __call__(self, ra, rb):
        """
        Note: 
        ra should be the positive particle and 
        rb should be the negative particle
        """
        ra = list(ra)
        rb = list(rb)

        args = ra + rb + self.parameters

        result = [dipole_f(*args) for dipole_f in self.dipole_function]
        
        return result

    def gradient(self,ra,rb,sign = "+"):
        ra = list(ra)
        rb = list(rb)
        args = ra + rb + self.parameters
        result = np.zeros((3,3))
        
        if sign == "+": 
            J_func = self.Ja_func
        elif sign == "-": 
            J_func = self.Jb_func
            
        for i, dmu_dRa in enumerate(J_func):
            for j, dmu_i_dRa in enumerate(dmu_dRa):
                result[i,j] = dmu_i_dRa(*args)

        return result

class GriegorievDipoleFunction(BaseDipoleFunction):
    def __init__(self, mu0, a, d0, d7):

        sm_d = ((sm_Rax - sm_Rbx)**2 + (sm_Ray - sm_Rby)**2 \
             + (sm_Raz - sm_Rbz)**2)**(1/2)
        
        self.parameters = [mu0, a, d0, d7]
        self.parameters_sm = [sm_mu0, sm_a, sm_d0, sm_d7]
        self.dipole_exp = [
            ((sm_Ra[i] - sm_Rb[i])/sm_d) *\
            (sm_mu0 * sm.exp(-sm_a*(sm_d-sm_d0)) - sm_d7/(sm_d**7))
            for i in range(3)
        ]
        
        super().__init__()

dipole_function1 = GriegorievDipoleFunction(mu0=0.0284, a=1.22522, d0=7.10, d7=14200)

print(dipole_function1([0,0,0],[5,0,0]))
print(dipole_function1.gradient([0,0,0],[5,0,0],"+"))
print(dipole_function1.gradient([5,0,0],[0,0,0],"-"))

[-0.19041015111388537, 0.0, 0.0]
[[-0.20152631  0.          0.        ]
 [ 0.          0.03808203  0.        ]
 [ 0.          0.          0.03808203]]
[[ 0.20152631  0.          0.        ]
 [ 0.         -0.03808203  0.        ]
 [ 0.          0.         -0.03808203]]


\begin{align*}
\dot{\textbf{C}}_k + i\omega \textbf{C}_k &= \frac{ie^{-i\textbf{k}\cdot \textbf{r}_\mu} }{\omega} \left(I - \frac{\textbf{k} \textbf{k}^T}{|\textbf{k}|} \right) \left( \frac{\partial \mu}{\partial \textbf{r}_{Ar} } \dot{\textbf{r}}_{Ar} + \frac{\partial \mu}{\partial \textbf{r}_{Xe} } \dot{\textbf{r}}_{Xe} \right)
\\
\omega &= ck
\end{align*}

In [22]:
r_ar = np.array([2,2,2]) # Argon should be negative
r_xe = np.array([0,0,0]) # Xenon should be positive
r = np.vstack([r_ar, r_xe])

v_ar = np.array([-1,-1,-1])
v_xe = np.array([0,0,0])
v = np.vstack([v_ar, v_xe])

Z_Ar = 18
m_Ar = Z_Ar * (m_p / m_e) + Z_Ar + (40 - Z_Ar) * (m_n / m_e) 
Z_Xe = 54
m_Xe = Z_Xe * (m_p / m_e) + Z_Xe + (131 - Z_Xe) * (m_n / m_e)

k_val = 1/constants.c
k_vec = np.array([1,0,0]) * k_vec 
epsilon_vec = np.array([[0,1,0], [0,0,1]])

L = 10

omega = constants.c * k_val
C = np.random.rand(2) + 1j * np.random.rand(2)

dipole_func = GriegorievDipoleFunction(mu0=0.0284, a=1.22522, d0=7.10, d7=14200)

pure_epsilon = np.array([0.996, 1.904]) * 1.59360e-3
mixed_epsilon = 1.377 * 1.59360e-3

pure_sigma = np.array([3.41, 4.06]) * (1e-10 / 5.29177e-11)
mixed_sigma = 3.735 * (1e-10 / 5.29177e-11)

epsilon = construct_param_matrix(2,1,pure_epsilon,mixed_epsilon)
sigma = construct_param_matrix(2,1,pure_sigma,mixed_sigma)

lennardj = LennardJonesPotential(
    n_points = 2,
    epsilon = epsilon,
    sigma = sigma,
    L = 10)

In [27]:
mu = dipole_func(r_xe, r_ar)

mu_grad_ar = dipole_func.gradient(r_ar, r_xe, "-")
mu_grad_xe = dipole_func.gradient(r_ar, r_xe, "+")
mu_grad = [mu_grad_ar, mu_grad_xe]

def dot_C(r, v, mu_grad, C, k_vec, omega, epsilon_vec):
    
    jk_mu = 0
    for i, ri in enumerate(r):
        jk_mu +=  np.exp(-1j * k_vec @ ri) * mu_grad[i].T @ v[i] 

    jk_mu *= (1j / omega)
    jktransv_mu = (np.eye(3) - np.outer(k_vec,k_vec) / (k_vec @ k_vec)) @ jk_mu #consider remove 

    proj_jktransv_mu = np.array([epsilon_vec[i]@jktransv_mu for i in range(len(epsilon_vec))])
    
    return proj_jktransv_mu - 1j * omega * C

dot_C(r,v,mu_grad,C,k_vec,omega,epsilon_vec)

array([2.53455524-1.1726274j , 2.15200071-1.21406872j])

\begin{align*}
    m_{Ar} \ddot{\textbf{r}}_{Ar}^{[\textbf{k}]} &= \sum_{i\neq \textbf{k}} \sum_{j \neq \textbf{k}} \dot{\textbf{r}}_{Ar}^{[i]} \frac{\partial \textbf{A}^{[j]} (\textbf{r}_\mu) }{\partial \textbf{r}^{[\textbf{k}]}_{Ar}}  \frac{\partial \mu^{[j]}}{\partial \textbf{r}_{Ar}^{[i]}} 
    - \sum_{j\neq\textbf{k}} \frac{\partial \textbf{A}^{[j]} (\textbf{r}_\mu) }{\partial t}  \frac{\partial \mu^{[j]}}{\partial \textbf{r}_{Ar}^{[\textbf{k}]}} 
    \\
    m_{Ar} \ddot{\textbf{r}}_{Ar}^{[\textbf{k}1]} 
    &= - \sum_j \frac{\partial \textbf{A}^{[j]} (\textbf{r}_\mu) }{\partial t}  \frac{\partial \mu^{[j]}}{\partial \textbf{r}_{Ar}^{[\textbf{k}1]}} 
    - \sum_{j\neq \textbf{k}} \dot{\textbf{r}}_{Ar}^{[\textbf{k}]} \frac{\partial \textbf{A}^{[j]} (\textbf{r}_\mu) }{\partial \textbf{r}_{Ar}^{[\textbf{k}]}} \frac{\partial \mu^{[j]}}{\partial \textbf{r}_{Ar}^{[\textbf{k}1]}}
\end{align*}

Note that, for simplicity, we currently choose:
\begin{align*}
    \mathbf{x} &= \mathbf{k}
    \\
    \mathbf{y} &= \mathbf{k1}
    \\
    \mathbf{z} &= \mathbf{k2}
\end{align*}

In [28]:
def compute_transv_force(r, v, mu_grad, C, C_dot, k_vec, omega, epsilon_vec):
    v_k = v @ k_vec
    v_kj = np.einsum("ij,j->i",epsilon_vec,v)
    k = np.sqrt(k_vec @ k_vec)

    ma = np.array([0+0j,0+0j,0+0j])

    for i in [0,1]:
        for j in [0,1]:
            ma[0] += (v_kj[i] * (1j * k * C[j] * np.exp(1j * k_vec @ r) \
                  + np.conjugate(1j * k * C[j] * np.exp(1j * k_vec @ r)) ) \
                  * mu_grad[i+1, j+1] # (grad mu)_ij = dmu_j / dr_i
                     ).ravel().item()

        ma[0] += (-mu_grad[0,i+1] * (1j * k * C_dot[i] * np.exp(1j * k_vec @ r) \
                      + np.conjugate(1j * k * C_dot[i] * np.exp(1j * k_vec @ r)) )
                 ).ravel().item()

    for i in [1,2]:
        for j in [0,1]:
            ma[i] += (-mu_grad[i,j+1] * (1j * k * C[j] * np.exp(1j * k_vec @ r) \
                          + np.conjugate(1j * k * C[j] * np.exp(1j * k_vec @ r)) )
                     ).ravel().item()
            ma[i] += (-v_k * mu_grad[j+1,i] * (1j * k * C_dot[j] * np.exp(1j * k_vec @ r) \
                                + np.conjugate(1j * k * C_dot[j] * np.exp(1j * k_vec @ r)) ) \
                     ).ravel().item()

    return ma

In [12]:
class Force:
    def __init__(self, potential, k_vec, omega, epsilon_vec):
        self.potential = potential
        self.k_vec = k_vec
        self.omega = omega
        self.epsilon_vec = epsilon_vec

    def dot_C(self, C):

    def __call__(self, r, v, mu_grad, C):
        repulsion = self.potential.get_force(r)
        force = []
        for i, ri in enumerate(r):
            force.append(
                compute_transv_force(ri, v[i], mu_grad[i], C, self.k_vec, self.omega) \
                + repulsion[i]
            )
        return np.array(force)

array([[ 89.20979848+0.j,  89.22820647+0.j,  89.22793574+0.j],
       [-89.22833895+0.j, -89.22832073+0.j, -89.22805046+0.j]])

In [9]:
get_force = Force(lennardj, k_vec, omega)

C_dot = dot_C
get_force(np.vstack([r_ar,r_xe]), np.vstack([v_ar,v_xe]), [mu_grad_ar,mu_grad_xe], C)

array([[ 89.23718511,  89.23718511,  89.23718511],
       [-89.23718511, -89.23718511, -89.23718511]])